In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = yf.download("PBW", start="2010-01-01", end="2022-03-19")

[*********************100%***********************]  1 of 1 completed


In [3]:
df = pd.DataFrame(data)

In [4]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,55.450001,56.750000,55.450001,56.750000,43.456409,107960
2010-01-05,56.849998,57.750000,56.450001,57.400002,43.954151,101740
2010-01-06,57.500000,58.549999,57.450001,58.250000,44.605038,181180
2010-01-07,58.250000,58.400002,57.700001,58.099998,44.490170,97420
2010-01-08,58.000000,58.799999,58.000000,58.799999,45.026203,75240


### Relative Strength Index (RSI)

In [5]:
delta = df['Close'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
df['RSI'] = round(100 - (100/(1 + rs)),2)

In [6]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,RSI
Date,,,,,,,
2010-01-04,55.450001,56.750000,55.450001,56.750000,43.456409,107960,NaN
2010-01-05,56.849998,57.750000,56.450001,57.400002,43.954151,101740,100.00
2010-01-06,57.500000,58.549999,57.450001,58.250000,44.605038,181180,100.00
2010-01-07,58.250000,58.400002,57.700001,58.099998,44.490170,97420,98.29
2010-01-08,58.000000,58.799999,58.000000,58.799999,45.026203,75240,98.43


### Stochastic Oscillator (slow,fast)

In [7]:
periods=14
    
high_roll = df["High"].rolling(periods).max()
low_roll = df["Low"].rolling(periods).min()
    
# Fast stochastic indicator
num = df["Close"] - low_roll
denom = high_roll - low_roll
df["%K"] = round((num / denom) * 100,2)
    
# Slow stochastic indicator
df["%D"] = round(df["%K"].rolling(3).mean(),2)

In [8]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,RSI,%K,%D
Date,,,,,,,,,
2010-01-04,55.450001,56.750000,55.450001,56.750000,43.456409,107960,NaN,NaN,NaN
2010-01-05,56.849998,57.750000,56.450001,57.400002,43.954151,101740,100.00,NaN,NaN
2010-01-06,57.500000,58.549999,57.450001,58.250000,44.605038,181180,100.00,NaN,NaN
2010-01-07,58.250000,58.400002,57.700001,58.099998,44.490170,97420,98.29,NaN,NaN
2010-01-08,58.000000,58.799999,58.000000,58.799999,45.026203,75240,98.43,NaN,NaN


### Advance/Decline Line

In [9]:
Advances = df['Close'] - df['Open']
Declines = df['Open'] - df['Close'].shift(1)

In [10]:
Cum_Advances = Advances.cumsum()
Cum_Declines = Declines.cumsum()

In [13]:
df['AD_Line'] = round(Cum_Advances - Cum_Declines,2)

In [14]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,RSI,%K,%D,AD_Line
Date,,,,,,,,,,
2010-01-04,55.450001,56.750000,55.450001,56.750000,43.456409,107960,NaN,NaN,NaN,NaN
2010-01-05,56.849998,57.750000,56.450001,57.400002,43.954151,101740,100.00,NaN,NaN,1.75
2010-01-06,57.500000,58.549999,57.450001,58.250000,44.605038,181180,100.00,NaN,NaN,2.40
2010-01-07,58.250000,58.400002,57.700001,58.099998,44.490170,97420,98.29,NaN,NaN,2.25
2010-01-08,58.000000,58.799999,58.000000,58.799999,45.026203,75240,98.43,NaN,NaN,3.15
